**1. Geliştirme Ortamının Hazırlanması ve Gerekli Kütüphanelerin Kurulumu**

Bu adım, RAG mimarisini kurmak, LLM'e bağlanmak ve web arayüzü oluşturmak için gereken tüm Python kütüphanelerini yükler. Projenin uyumluluğunu sağlamak amacıyla, LangChain paketleri (langchain, langchain-core, langchain-community) ve tünelleme araçları (pycloudflared) gibi temel bağımlılıklar yeniden yüklenmiştir.

**Gerekli Kütüphaneler:**

* **RAG Pipeline Framework:** langchain

* **LLM (Generation):** langchain-google-genai (Gemini API için)

* **Embedding Model:** sentence-transformers

* **Vektör Veritabanı:** chromadb

* **Veri Okuma:** pypdf

* **Web Arayüzü:** streamlit

Çalıştırma Notu: Bu bloğun çalıştırılmasının ardından, kurulumun tam olarak etkinleşmesi için Çalışma Zamanı > Oturumu Yeniden Başlat yapılmalıdır.

In [1]:
# Adım 1: Uyumsuzluk yaratan tüm LangChain paketlerini kaldırıyoruz
!pip uninstall -y langchain langchain-community langchain-core langchain-google-genai

# Adım 2: Gerekli kütüphaneleri yeniden yüklüyoruz (En güncel ve uyumlu versiyonlar)
!pip install -q transformers torch
!pip install -q pypdf
!pip install -q chromadb
!pip install -q sentence-transformers
!pip install -q streamlit
!pip install -q langchain-google-genai
!pip install -q langchain langchain-community langchain-core

print("Tüm gerekli kütüphaneler başarıyla yüklendi.")

# ÖNEMLİ: Oturumu yeniden başlatıyoruz ki yeni kütüphane yolları etkinleşsin.
print("Ortamın etkinleşmesi için lütfen Çalışma Zamanı > Oturumu Yeniden Başlat yapın.")

Found existing installation: langchain 1.0.2
Uninstalling langchain-1.0.2:
  Successfully uninstalled langchain-1.0.2
Found existing installation: langchain-community 0.4
Uninstalling langchain-community-0.4:
  Successfully uninstalled langchain-community-0.4
Found existing installation: langchain-core 1.0.0
Uninstalling langchain-core-1.0.0:
  Successfully uninstalled langchain-core-1.0.0
Found existing installation: langchain-google-genai 3.0.0
Uninstalling langchain-google-genai-3.0.0:
  Successfully uninstalled langchain-google-genai-3.0.0
Tüm gerekli kütüphaneler başarıyla yüklendi.
Ortamın etkinleşmesi için lütfen Çalışma Zamanı > Oturumu Yeniden Başlat yapın.


In [2]:
# Yeni ve eksik paketi yüklüyoruz
!pip install -q langchain-text-splitters

**2. Veri Seti Hazırlama ve RAG Pipeline'ının Kurulması**

Bu aşama, RAG mimarisinin (Retrieval Augmented Generation) temelini oluşturur.

1. **API Anahtarı Yönetimi:** Gemini API Anahtarı, güvenlik amacıyla Colab'ın Gizli Anahtarlarından (userdata) çekilir ve ortam değişkenine ayarlanır.

2. **Veri Okuma ve Parçalama (Chunking):** Proje dosyası (GenAI Bootcamp Proje Dosyası.pdf) okunur ve metin, modelin daha iyi anlaması için 1000 karakterlik parçalara (langchain-text-splitters) ayrılır.

3. **Vektörleştirme (Embedding):** Parçalar, Türkçe desteği olan paraphrase-multilingual-MiniLM-L12-v2 modeli kullanılarak vektörlere dönüştürülür.

4. **Vektör Veritabanı:** Bu vektörler, hızlı arama için ChromaDB'ye kaydedilir ve bir Retriever (bilgi çekici) oluşturulur.

In [6]:
import os
from google.colab import userdata
from pypdf import PdfReader
# DÜZELTİLMİŞ İMPORT: Artık 'langchain_text_splitters' paketinden çekiyoruz!
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# --- 2.1 API Anahtarını Çekme ---
try:
    os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
    print("Gemini API Anahtarı, ortam değişkeni olarak ayarlandı.")
except userdata.SecretNotFoundError:
    raise ValueError("HATA: 'GEMINI_API_KEY' bulunamadı. Lütfen kontrol edin.")

# --- 2.2 PDF'i Okuma ---
pdf_path = "GenAI Bootcamp Proje Dosyası.pdf"
reader = PdfReader(pdf_path)
raw_text = ''.join(page.extract_text() + "\n" for page in reader.pages if page.extract_text())
print(f"Toplam {len(reader.pages)} sayfa okundu.")

# --- 2.3 Veri Parçalama (Chunking) ---
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)
texts = text_splitter.create_documents([raw_text])
print(f"Oluşturulan toplam parça (chunk) sayısı: {len(texts)}")

# --- 2.4 Vektörleştirme ve Kayıt ---
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

vectorstore = Chroma.from_documents(documents=texts, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print("Vektörleştirme ve Retriever başarıyla tamamlandı.")

Gemini API Anahtarı, ortam değişkeni olarak ayarlandı.
Toplam 3 sayfa okundu.
Oluşturulan toplam parça (chunk) sayısı: 5


/tmp/ipython-input-539952497.py:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vektörleştirme ve Retriever başarıyla tamamlandı.


**3. RAG Zincirinin Kurulumu ve Fonksiyonel Test**

Bu adım, Retrieval (bilgi çekme) ve Generation (cevap üretme) aşamalarını birleştirir.

1. **LLM Yükleme:** Gemini 2.5 Flash modeli, doğrudan API Anahtarı (api_key) parametresi kullanılarak başlatılır.

2. **Prompt Mühendisliği:** LLM'e sadece sağlanan bağlamı (context) kullanması ve bilgi eksikliğinde bunu belirtmesi talimatı verilir.

3. **Zincir Kurulumu (LCEL):** LangChain Expression Language (LCEL) kullanılarak RAG akışı oluşturulur (rag_chain).

4. **Test:** Kurulan zincir, rastgele bir proje kuralı ile test edilir.

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os # Ortam değişkenini okumak için tekrar import ediyoruz.

# 0. API Anahtarını Ortamdan Çekme (Hata almamak için koda da ekleyelim)
# Bu satır, Colab secrets'tan değeri okur.
gemini_api_key = os.environ.get('GEMINI_API_KEY')

# 1. LLM Modelini Yükle
# DÜZELTME: API anahtarını doğrudan 'api_key' parametresi ile geçiriyoruz.
if not gemini_api_key:
    raise ValueError("Gemini API Anahtarı bulunamadı. Lütfen kontrol edin.")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
    api_key=gemini_api_key # Anahtarı doğrudan buraya geçiriyoruz!
)

# 2. Prompt (İstek Metni) Tanımlama
template = """Sen, Global AI Hub/Akbank GenAI Bootcamp Proje ve Kural Asistanısın.
Yalnızca sağlanan bağlam (context) bilgisine dayanarak kullanıcı sorularını cevapla.
Eğer bağlamda yeterli bilgi yoksa, "Veri setimde bu konuyla ilgili yeterli bilgi bulunmamaktadır." diye cevap ver.

Bağlam (Context): {context}
Soru: {question}
Cevap:
"""
prompt = ChatPromptTemplate.from_template(template)

# 3. RAG Zincirini Kurma (rag_chain)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG Pipeline (rag_chain) başarıyla kuruldu ve test aşamasına hazır.")

# --- TEST KISMI ---
query = "Proje değerlendirme kriterleri nelerdir?"
response = rag_chain.invoke(query)

print("\n--- CHATBOT CEVABI ---")
print(response)

RAG Pipeline (rag_chain) başarıyla kuruldu ve test aşamasına hazır.

--- CHATBOT CEVABI ---
Proje değerlendirme kriterleri şunlardır:
*   Geliştirme Ortamı (Github & README.md)
*   Veri Seti Hazırlama
*   Çalışma Kılavuzu
*   Çözüm mimariniz
*   Web Arayüzü & Product Kılavuzu


**4. Web Arayüzü (Streamlit) Uygulamasının Oluşturulması** (app.py)
Bu adım, projenin **Web Arayüzü ve Product Kılavuzu** gereksinimini karşılar. app.py dosyası, Streamlit kütüphanesi kullanılarak oluşturulur.

* **Mimari:** Tüm RAG kurulumu @st.cache_resource decorator'ı ile önbelleğe alınır, böylece uygulama her yenilendiğinde tekrar yüklenmez.

* **Güvenlik Çözümü:** Streamlit uygulamasının arka planda çalışırken API anahtarını okuyamaması hatasını çözmek için, anahtarın GEMINI_API_KEY_TEMP dosyasından okunması sağlanmıştır.

* **Kullanıcı Arayüzü:** Basit bir başlık, açıklama ve kullanıcıdan girdi almak için bir metin kutusu (st.text_input) içerir.

In [12]:
import time # EKLENMELİ: Bu hata için gerekli olan satır!
from google.colab import userdata # Bu import zaten olmalı
import os # Bu import da olmalı
import subprocess # Bu import da olmalı

# 1. Tüm eski süreçleri öldür (Tüneli kapatır)
!killall streamlit &>/dev/null
!killall cloudflared &>/dev/null
time.sleep(1)

# 2. API Anahtarını Geçir ve Streamlit'i Başlat
# ... (kodun geri kalanı)

In [8]:
%%writefile app.py

import streamlit as st
import os
import shutil
# Gerekli RAG Kütüphaneleri
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from google.colab import userdata # API Anahtarını çekmek için


# --- RAG SİSTEMİ KURULUMU (Önbellekte Tutulur) ---
@st.cache_resource
def setup_rag_system():
    # API Anahtarını geçici dosyadan okuma
    api_key_path = "GEMINI_API_KEY_TEMP"
    if os.path.exists(api_key_path):
        with open(api_key_path, 'r') as f:
            gemini_api_key = f.read().strip()
    else:
        st.error("KRİTİK HATA: API Anahtarı Streamlit ortamında bulunamadı. Lütfen Colab'da Adım 2'yi kontrol edin.")
        return None

    # --- Diğer RAG kurulum kodlarınız buraya gelir (Kod Bloğu 2 ile aynı mantık) ---
    pdf_path = "GenAI Bootcamp Proje Dosyası.pdf"
    if not os.path.exists(pdf_path):
        st.error(f"HATA: '{pdf_path}' dosyası bulunamadı. Lütfen Colab'a tekrar yükleyin.")
        return None

    reader = PdfReader(pdf_path)
    raw_text = ''.join(page.extract_text() + "\n" for page in reader.pages if page.extract_text())
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)
    texts = text_splitter.create_documents([raw_text])

    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    model_kwargs = {'device': 'cuda'}
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
    vectorstore = Chroma.from_documents(documents=texts, embedding=embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    # RAG Zincirini Kurma
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2, api_key=gemini_api_key)

    # KRİTİK DÜZELTME: PROMPT METNİ EKLENDİ
    template = """Sen, Global AI Hub/Akbank GenAI Bootcamp Proje ve Kural Asistanısın.
    Yalnızca sağlanan bağlam (context) bilgisine dayanarak kullanıcı sorularını cevapla.
    Eğer bağlamda yeterli bilgi yoksa, "Veri setimde bu konuyla ilgili yeterli bilgi bulunmamaktadır." diye cevap ver.

    Bağlam (Context): {context}
    Soru: {question}
    Cevap:
    """
    prompt = ChatPromptTemplate.from_template(template)
    rag_chain = ({"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser())

    return rag_chain

# --- STREAMLIT ARAYÜZÜ ---
st.set_page_config(page_title="Bootcamp Proje Asistanı", layout="wide")
st.title("✨ GenAI Bootcamp Proje ve Kural Asistanı")
st.markdown("Merhaba! Bana projeniz, kurallar, veri setleri veya teslimat hakkında sorular sorabilirsiniz.")

# Yorum satırını kaldırıyoruz ki, web sitesindeki hata mesajı yerine RAG sistemi kurulsun.
rag_chain = setup_rag_system()

if rag_chain:
    query = st.text_input("Sorunuzu buraya yazın:", placeholder="README.md dosyasında neler olmalı?")
    if query:
        with st.spinner(f"'{query}' için cevap aranıyor..."):
            try:
                response = rag_chain.invoke(query)
                st.info(response)
            except Exception as e:
                st.error(f"Chatbot bir hata ile karşılaştı: {e}")

Writing app.py


**5. Web Arayüzü Dağıtımı (Deployment) ve Canlı Bağlantı**

Bu adım, Web Arayüzü & Product Kılavuzu gereksinimini karşılar.

**Süreç Temizliği:** Önceki denemelerden kalan tüm arka plan süreçleri (streamlit, cloudflared) öldürülerek temiz bir başlangıç sağlanır.

**Güvenli Anahtar Transferi:** API anahtarı, Streamlit'in arka plan süreçlerinde okuyabilmesi için Colab'dan çekilir ve geçici bir dosyaya (GEMINI_API_KEY_TEMP) yazılır.

**Streamlit Başlatma:** Uygulama arka planda (&>/dev/null&) yeniden başlatılır ve hazır olması için bekleme süresi tanınır.

**Tünelleme:** Projeyi dış dünyaya açmak için, güvenilir bir araç olan Cloudflare Argo Tunnel (cloudflared) terminal komutu ile çalıştırılır.

**Sonuç:** Komutun çıktısı olarak üretilen URL (link), projenin **teslim linkidir** ve **README.md** dosyasında paylaşılmalıdır.

In [13]:
# 1. Tüm eski süreçleri öldür (Tüneli kapatır)
!killall streamlit &>/dev/null
!killall cloudflared &>/dev/null
time.sleep(1)

# 2. API Anahtarını Geçir ve Streamlit'i Başlat
gemini_api_key = userdata.get('GEMINI_API_KEY')
api_key_path = "GEMINI_API_KEY_TEMP"

with open(api_key_path, 'w') as f:
    f.write(gemini_api_key)
print(f"API Anahtarı, Streamlit'in okuyabileceği {api_key_path} dosyasına yeniden yazıldı.")

print("Streamlit uygulamasını yeniden başlatılıyor...")
!streamlit run app.py --server.port 8501 --server.enableCORS false &>/dev/null&
time.sleep(10)

# 3. Cloudflare Tünelini Yeniden Aç
print("Cloudflare Argo Tunnel (cloudflared terminal komutu ile) kuruluyor...")
!wget -q -O cloudflared "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"
!chmod +x cloudflared
time.sleep(5)

!./cloudflared tunnel --url http://localhost:8501

API Anahtarı, Streamlit'in okuyabileceği GEMINI_API_KEY_TEMP dosyasına yeniden yazıldı.
Streamlit uygulamasını yeniden başlatılıyor...
Cloudflare Argo Tunnel (cloudflared terminal komutu ile) kuruluyor...
2025-10-22T16:36:33Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-22T16:36:33Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-22T16:36:36Z INF +------------------------------------------------------------------------------------------

In [14]:
# requirements.txt dosyasını oluşturma
!pip freeze > requirements.txt
print("requirements.txt dosyası oluşturuldu.")

requirements.txt dosyası oluşturuldu.
